# <center>JSONiq Update Facility sandbox</center>


The following cell is meant for Google Colab, which does not have the right Java version.

In [ ]:
# We make sure Java 17 is installed.
!apt update
!apt install openjdk-17-jdk
!java -version

In [1]:
!pip install jsoniq


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


We now initialize a RumbleDB session and test that it works.

In [2]:
from jsoniq import RumbleSession

rumble = RumbleSession.builder.withDelta().getOrCreate();

print(rumble.jsoniq('{ "foo": [ 6*7 ] }').json());

:: loading settings :: url = jar:file:/Users/ghislain/.pyenv/versions/3.11.9/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/ghislain/.ivy2.5.2/cache
The jars for the packages stored in: /Users/ghislain/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-036ada57-26e0-4a3f-90ea-b37f8580587f;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in local-m2-cache
	found io.delta#delta-storage;4.0.0 in local-m2-cache
	found org.antlr#antlr4-runtime;4.13.1 in local-m2-cache
:: resolution report :: resolve 68ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from local-m2-cache in [default]
	io.delta#delta-storage;4.0.0 from local-m2-cache in [default]
	org.antlr#antlr4-runtime;4.13.1 from local-m2-cache in [default]
	---------------------------------------------------------------------
	|         

({'foo': [42]},)


For convenience, we also activate the JSONiq magic.

In [3]:
%load_ext jsoniqmagic


JavaObject id=o46

We create a new collection

In [5]:
%%jsoniq -u -t
create collection table("foo") with for $i in 1000000 to 3000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }


Updates applied successfully.
Response time: 3.293159008026123 ms


In [6]:
%%jsoniq -df -t
table("foo")

+-------+--------------------+-------+----------+-----------------+--------+---------------+
|    foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+-------+--------------------+-------+----------+-----------------+--------+---------------+
|1000000|[Alice, Bob, Char...|      0|       0.0|                0|        |            foo|
|1000001|[Alice, Bob, Char...|      1|       1.0|                0|        |            foo|
|1000002|[Alice, Bob, Char...|      2|       2.0|                0|        |            foo|
|1000003|[Alice, Bob, Char...|      3|       3.0|                0|        |            foo|
|1000004|[Alice, Bob, Char...|      4|       4.0|                0|        |            foo|
|1000005|[Alice, Bob, Char...|      5|       5.0|                0|        |            foo|
|1000006|[Alice, Bob, Char...|      6|       6.0|                0|        |            foo|
|1000007|[Alice, Bob, Char...|      7|       7.0|                0|   

In [7]:
%%jsoniq -df -t
count(table("foo"))

+-------+
|  value|
+-------+
|2000001|
+-------+

Response time: 0.18091177940368652 ms


We insert new data -- remember the snapshot semantics, all updates are computed against the original table, and then applied jointly.

In [8]:
%%jsoniq -u -t
insert for $i in 4000000 to 6000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  first into collection table("foo"),
insert for $i in 7000000 to 10000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  last into collection table("foo"),
insert for $i in 11000000 to 13000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  before table("foo")[position() eq 2] into collection,
insert for $i in 14000000 to 16000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  after table("foo")[position() eq 2] into collection


Updates applied successfully.
Response time: 13.836093187332153 ms


In [9]:
%%jsoniq -df -t
table("foo")

+-------+--------------------+-----------+----------+-----------------+--------+---------------+
|    foo|                 bar|    __rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+-------+--------------------+-----------+----------+-----------------+--------+---------------+
|4000000|[Alice, Bob, Char...|17183869187| -99999.95|                0|        |            foo|
|4000001|[Alice, Bob, Char...|17183869188|  -99999.9|                0|        |            foo|
|4000002|[Alice, Bob, Char...|17183869189| -99999.85|                0|        |            foo|
|4000003|[Alice, Bob, Char...|17183869190|  -99999.8|                0|        |            foo|
|4000004|[Alice, Bob, Char...|17183869191| -99999.75|                0|        |            foo|
|4000005|[Alice, Bob, Char...|17183869192|  -99999.7|                0|        |            foo|
|4000006|[Alice, Bob, Char...|17183869193| -99999.65|                0|        |            foo|
|4000007|[Alice, Bob, Char...|

In [10]:
%%jsoniq -df -t
count(table("foo"))

+--------+
|   value|
+--------+
|11000005|
+--------+

Response time: 0.10588693618774414 ms


We delete a record

In [11]:
%%jsoniq -u -t
delete table("foo")[$$.foo eq 1500000] from collection

Updates applied successfully.
Response time: 1.3398218154907227 ms


In [12]:
%%jsoniq -df -t
count(table("foo"))

+--------+
|   value|
+--------+
|11000004|
+--------+

Response time: 0.22942113876342773 ms


We edit a record

In [13]:
%%jsoniq -u -t
edit table("foo")[$$.foo eq 1400000] into { "foo" : 22170000, "bar" : [ "Alice", "Bob", "Daniel" ] } in collection

Updates applied successfully.
Response time: 1.5862219333648682 ms


In [14]:
%%jsoniq
table("foo")[$$.foo eq 22170000]

{
  "foo": 22170000,
  "bar": [
    "Alice",
    "Bob",
    "Daniel"
  ]
}


In [15]:
%%jsoniq -df -t
table("foo")

+-------+--------------------+-----------+----------+-----------------+--------+---------------+
|    foo|                 bar|    __rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+-------+--------------------+-----------+----------+-----------------+--------+---------------+
|4000000|[Alice, Bob, Char...|17183869187| -99999.95|                0|        |            foo|
|4000001|[Alice, Bob, Char...|17183869188|  -99999.9|                0|        |            foo|
|4000002|[Alice, Bob, Char...|17183869189| -99999.85|                0|        |            foo|
|4000003|[Alice, Bob, Char...|17183869190|  -99999.8|                0|        |            foo|
|4000004|[Alice, Bob, Char...|17183869191| -99999.75|                0|        |            foo|
|4000005|[Alice, Bob, Char...|17183869192|  -99999.7|                0|        |            foo|
|4000006|[Alice, Bob, Char...|17183869193| -99999.65|                0|        |            foo|
|4000007|[Alice, Bob, Char...|

In [16]:
%%jsoniq -t
rename json table("foo")[1].foo as "bar2";

Response time: 3.707296133041382 ms


In [17]:
%%jsoniq -df -t
table("foo")

+-------+--------------------+-----------+----------+-------+-----------------+--------+---------------+
|    foo|                 bar|    __rowID|__rowOrder|   bar2|__mutabilityLevel|__pathIn|__tableLocation|
+-------+--------------------+-----------+----------+-------+-----------------+--------+---------------+
|   NULL|[Alice, Bob, Char...|17183869187| -99999.95|4000000|                0|        |            foo|
|4000001|[Alice, Bob, Char...|17183869188|  -99999.9|   NULL|                0|        |            foo|
|4000002|[Alice, Bob, Char...|17183869189| -99999.85|   NULL|                0|        |            foo|
|4000003|[Alice, Bob, Char...|17183869190|  -99999.8|   NULL|                0|        |            foo|
|4000004|[Alice, Bob, Char...|17183869191| -99999.75|   NULL|                0|        |            foo|
|4000005|[Alice, Bob, Char...|17183869192|  -99999.7|   NULL|                0|        |            foo|
|4000006|[Alice, Bob, Char...|17183869193| -99999.65|  

In [18]:
%%jsoniq -u -t
delete collection table("foo")

Updates applied successfully.
Response time: 0.035366058349609375 ms


In [28]:
%%jsoniq -u -t
create collection delta-file("foobar") with for $i in 1000000 to 3000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }


Updates applied successfully.
Response time: 0.905670166015625 ms


In [29]:
%%jsoniq -df -t
delta-file("foobar")

+-------+--------------------+-------+----------+-----------------+--------+--------------------+
|    foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+-------+--------------------+-------+----------+-----------------+--------+--------------------+
|1000000|[Alice, Bob, Char...|      0|       0.0|                0|        |delta.`file:/User...|
|1000001|[Alice, Bob, Char...|      1|       1.0|                0|        |delta.`file:/User...|
|1000002|[Alice, Bob, Char...|      2|       2.0|                0|        |delta.`file:/User...|
|1000003|[Alice, Bob, Char...|      3|       3.0|                0|        |delta.`file:/User...|
|1000004|[Alice, Bob, Char...|      4|       4.0|                0|        |delta.`file:/User...|
|1000005|[Alice, Bob, Char...|      5|       5.0|                0|        |delta.`file:/User...|
|1000006|[Alice, Bob, Char...|      6|       6.0|                0|        |delta.`file:/User...|
|1000007|[Alice, Bob

In [30]:
%%jsoniq -u -t
insert for $i in 4000000 to 6000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  first into collection delta-file("foobar"),
insert for $i in 7000000 to 10000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  last into collection delta-file("foobar"),
insert for $i in 11000000 to 13000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  before delta-file("foobar")[position() eq 2] into collection,
insert for $i in 14000000 to 16000000 return { "foobar" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  after delta-file("foobar")[position() eq 2] into collection


Updates applied successfully.
Response time: 12.24434494972229 ms


In [31]:
%%jsoniq -df -t
delta-file("foobar")

+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|    foo|                 bar|    __rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|4000000|[Alice, Bob, Char...|17183869187| -99999.95|                0|        |delta.`file:/User...|
|4000001|[Alice, Bob, Char...|17183869188|  -99999.9|                0|        |delta.`file:/User...|
|4000002|[Alice, Bob, Char...|17183869189| -99999.85|                0|        |delta.`file:/User...|
|4000003|[Alice, Bob, Char...|17183869190|  -99999.8|                0|        |delta.`file:/User...|
|4000004|[Alice, Bob, Char...|17183869191| -99999.75|                0|        |delta.`file:/User...|
|4000005|[Alice, Bob, Char...|17183869192|  -99999.7|                0|        |delta.`file:/User...|
|4000006|[Alice, Bob, Char...|17183869193| -99999.65|                0|        |de

In [32]:
%%jsoniq -u -t
delete delta-file("foobar")[$$.foo eq 15000000] from collection

Updates applied successfully.
Response time: 1.5392799377441406 ms


In [33]:
%%jsoniq -df -t
delta-file("foobar")

+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|    foo|                 bar|    __rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|4000000|[Alice, Bob, Char...|17183869187| -99999.95|                0|        |delta.`file:/User...|
|4000001|[Alice, Bob, Char...|17183869188|  -99999.9|                0|        |delta.`file:/User...|
|4000002|[Alice, Bob, Char...|17183869189| -99999.85|                0|        |delta.`file:/User...|
|4000003|[Alice, Bob, Char...|17183869190|  -99999.8|                0|        |delta.`file:/User...|
|4000004|[Alice, Bob, Char...|17183869191| -99999.75|                0|        |delta.`file:/User...|
|4000005|[Alice, Bob, Char...|17183869192|  -99999.7|                0|        |delta.`file:/User...|
|4000006|[Alice, Bob, Char...|17183869193| -99999.65|                0|        |de

In [35]:
%%jsoniq -u -t
edit delta-file("foobar")[$$.foo eq 1400000] into { "foo" : 17000000, "bar" : [ "Alice", "Bob", "Daniel" ] } in collection

Updates applied successfully.
Response time: 1.647326946258545 ms


In [36]:
%%jsoniq -df -t
delta-file("foobar")

+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|    foo|                 bar|    __rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|4000000|[Alice, Bob, Char...|17183869187| -99999.95|                0|        |delta.`file:/User...|
|4000001|[Alice, Bob, Char...|17183869188|  -99999.9|                0|        |delta.`file:/User...|
|4000002|[Alice, Bob, Char...|17183869189| -99999.85|                0|        |delta.`file:/User...|
|4000003|[Alice, Bob, Char...|17183869190|  -99999.8|                0|        |delta.`file:/User...|
|4000004|[Alice, Bob, Char...|17183869191| -99999.75|                0|        |delta.`file:/User...|
|4000005|[Alice, Bob, Char...|17183869192|  -99999.7|                0|        |delta.`file:/User...|
|4000006|[Alice, Bob, Char...|17183869193| -99999.65|                0|        |de

In [37]:
%%jsoniq -u -t
delete collection delta-file("foobar")

Updates applied successfully.
Response time: 0.03163290023803711 ms
